In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install -q --upgrade pip
!pip install -q fastapi uvicorn nest-asyncio pydantic[dotenv]
!pip install -q streamlit requests httpx python-multipart
!pip install -q langchain langchain-community langchain-huggingface
!pip install -q pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 152.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 166.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [streamlit]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [langchain-community]


In [1]:
import os
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
import uvicorn, nest_asyncio, threading
from pyngrok import ngrok

from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

/tmp/ipython-input-1-310415437.py:6: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from huggingface_hub import login

HF_TOKEN = "YOUR HUGGINGFACE TOKEN"  # <-- REPLACE THIS
login(token=HF_TOKEN)

In [3]:
# Clear GPU memory
torch.cuda.empty_cache()

# Model configuration
MODEL_ID = "abdulsamad99/medical-fine-tuning"
MAX_SEQ_LENGTH = 512

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model, _ = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True
)
model = FastLanguageModel.for_inference(model).to("cuda")

print("✅ Model loaded successfully on GPU:", torch.cuda.get_device_name(0))


==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.7.5 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Model loaded successfully on GPU: Tesla T4


In [7]:
NGROK_TOKEN = "YOUR NGROK TOKEN"  # <-- your token
ngrok.set_auth_token(NGROK_TOKEN)

In [4]:
# ====== Per-session version (REPLACES global chat_history & chain) ======
import re, torch
from collections import defaultdict
from langchain.schema import HumanMessage, AIMessage
from langchain_core.runnables import RunnableSequence, RunnableLambda

SESSION_HISTORIES = defaultdict(list)  # session_id -> list[Messages]

RE_THINK = re.compile(r"<think>(.*?)</think>", re.DOTALL | re.IGNORECASE)
RE_RESPONSE = re.compile(r"<response>(.*?)</response>", re.DOTALL | re.IGNORECASE)

def format_history(session_id: str, max_turns: int = 5) -> str:
    msgs = SESSION_HISTORIES[session_id]
    if max_turns:
        msgs = msgs[-(max_turns*2):]
    lines = []
    for m in msgs:
        if isinstance(m, HumanMessage):
            lines.append(f"Doctor: {m.content.strip()}")
        else:
            lines.append(f"Assistant: {m.content.strip()}")
    return " | ".join(lines)

def build_prompt_from_inputs(inputs: dict) -> str:
    session_id = inputs["session_id"]
    symptom = inputs["symptom"]
    ctx = format_history(session_id, max_turns=3)
    ctx_line = f"Context: {ctx}\n" if ctx else ""
    return f"{ctx_line}Question: {symptom}\n<think>"

def llama_generate(prompt: str) -> str:
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH).to("cuda")
    input_len = enc["input_ids"].shape[1]
    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=450,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
    gen_tokens = out[0, input_len:]
    return tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

def run_model_and_merge(prompt: str) -> str:
    gen = llama_generate(prompt)
    return prompt + gen

def parse_full_text(full_text: str) -> dict:
    think_m = RE_THINK.search(full_text)
    resp_m = RE_RESPONSE.search(full_text)
    think = think_m.group(1).strip() if think_m else ""
    diagnosis = resp_m.group(1).strip() if resp_m else full_text.strip()
    return {"think": think, "diagnosis": diagnosis}

llm_chain = RunnableSequence(
    RunnableLambda(build_prompt_from_inputs)
    | RunnableLambda(run_model_and_merge)
    | RunnableLambda(parse_full_text)
)
print("✅ Per-session chain ready.")


✅ Per-session chain ready.


In [5]:
# ================= FastAPI (Option B: per-session) =================
import nest_asyncio, threading, uvicorn, time
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok

app = FastAPI(title="Medical Diagnosis API (Per Session)")

class SymptomRequest(BaseModel):
    symptom: str
    session_id: str

@app.post("/diagnose")
async def diagnose(req: SymptomRequest):
    try:
        SESSION_HISTORIES[req.session_id].append(HumanMessage(content=req.symptom))
        result = llm_chain.invoke({"session_id": req.session_id, "symptom": req.symptom})
        SESSION_HISTORIES[req.session_id].append(AIMessage(content=result["diagnosis"]))
        return {"diagnosis": result["diagnosis"], "think": result["think"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

threading.Thread(target=run_api, daemon=True).start()
time.sleep(2)
public_url = ngrok.connect(8000).public_url
print("FastAPI running:", public_url)


INFO:     Started server process [7084]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FastAPI running: https://0d6b4b562202.ngrok-free.app


In [6]:
# ================= Streamlit (Option B) =================
with open("app.py", "w") as f:
    f.write(f"""
import streamlit as st
import requests, os

API_URL = "{public_url}/diagnose"

st.set_page_config(page_title="Medical Diagnosis Assistant", page_icon="🩺")
st.title("🩺 Medical Diagnosis Assistant (Per Session)")

if "session_id" not in st.session_state:
    st.session_state.session_id = os.urandom(8).hex()
if "chat" not in st.session_state:
    st.session_state.chat = []

symptoms = st.text_area("Enter symptoms / question:", height=120)
show_think = st.checkbox("Show reasoning (<think>)", value=False)

if st.button("Get Diagnosis"):
    if symptoms.strip():
        resp = requests.post(API_URL, json={{"symptom": symptoms, "session_id": st.session_state.session_id}})
        if resp.status_code == 200:
            data = resp.json()
            st.session_state.chat.append(("Doctor", symptoms))
            st.session_state.chat.append(("Assistant", data.get("diagnosis","[No diagnosis]"), data.get("think","")))
        else:
            st.error(f"Error {{resp.status_code}}: {{resp.text}}")
    else:
        st.warning("Please enter something first.")

st.subheader("Conversation")
for msg in st.session_state.chat:
    if msg[0] == "Doctor":
        st.markdown(f"**Doctor:** {{msg[1]}}")
    else:
        st.markdown(f"**Assistant:** {{msg[1]}}")
        if show_think and len(msg) > 2 and msg[2]:
            with st.expander("Reasoning"):
                st.write(msg[2])

col1, col2 = st.columns(2)
with col1:
    if st.button("New Session"):
        st.session_state.session_id = os.urandom(8).hex()
        st.session_state.chat = []
        st.success("Started a new session.")
with col2:
    if st.button("Clear Chat"):
        st.session_state.chat = []
""")

import subprocess, time
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])
time.sleep(5)
streamlit_url = ngrok.connect(8501).public_url
print("Streamlit running:", streamlit_url)


Streamlit running: https://e53d6b620d15.ngrok-free.app
